In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
train1 = pd.read_csv('../data/train1level.csv')
test1 = pd.read_csv('../data/test1level.csv')
holdout1 = pd.read_csv('../data/holdout1level.csv')

In [3]:
items = train1.movieid.unique()
test1 = test1[test1.movieid.isin(items)]
users_test = test1.userid.unique()
holdout1 = holdout1[holdout1.userid.isin(users_test)]

In [5]:
test1['rating'], rating_idx_map = pd.factorize(test1['rating'], sort=True)

In [6]:
rating_idx_map

Float64Index([0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0], dtype='float64')

In [7]:
test1['rating'].unique()

array([5, 3, 9, 0, 8, 7, 6, 4, 2, 1])

In [117]:
import omegaconf
cfg = omegaconf.OmegaConf.load('./config/config.yaml')

In [119]:
from src import TensorModel

tensor_model = TensorModel(**cfg.TensorModel)
tensor_model.fit(train1)
recommend_items, scores = tensor_model.get_recommendation(test1)

Step 1 of 5
growth of the core: 1.0
Step 2 of 5
growth of the core: 0.19415548308864405
Step 3 of 5
growth of the core: 0.01800236839212035
Step 4 of 5
growth of the core: 0.0030412665814473056
Step 5 of 5
growth of the core: 0.0009536208181636117
Core is no longer growing. Norm of the core: 1106.3709943960478.
Done


100%|██████████████████████████████████| 3996/3996 [00:18<00:00, 213.36it/s]


In [122]:
from src import get_metrics
get_metrics(recommend_items, holdout1, tensor_model.n_items)

{'hr': 0.004754754754754755,
 'mrr': 0.001363367335589558,
 'cov': 0.0010441683199331733}

In [3]:
%cd ..

/Users/a.bredikhin/PycharmProjects/RecSys-airi


In [8]:
from src import train_test_split, TwoLevelRecSystem, EASEModel
import omegaconf

cfg = omegaconf.OmegaConf.load('./config/config.yaml')

In [17]:
stage1_train, stage2_predict, stage2_train, stage2_holdout, final_training, final_testset, final_holdout = train_test_split(**cfg.dataset)
n_items = max(final_training.movieid) + 1

2022-07-18 23:59:12.171 | INFO     | src.preprocess:train_test_split:132 - start split dataset


Filtered 23447 invalid observations.


2022-07-18 23:59:19.926 | SUCCESS  | src.preprocess:train_test_split:144 - end split dataset


Filtered 20580 invalid observations.


In [334]:
model = TwoLevelRecSystem(n_items, cfg.EASEModel, cfg.TensorModel, {})

In [335]:
model.fit(stage1_train, stage2_predict, stage2_train)

2022-07-16 12:41:58.720 | DEBUG    | src.pipeline:fit_first_level_model:36 - Start fit ease model
2022-07-16 12:41:59.262 | INFO     | src.EASEModel:fit:18 - start fitting
2022-07-16 12:41:59.263 | INFO     | src.EASEModel:fit:20 - normalize
2022-07-16 12:41:59.529 | INFO     | src.EASEModel:fit:25 - gram matrix
2022-07-16 12:42:06.249 | INFO     | src.EASEModel:fit:32 - invert
2022-07-16 12:42:24.010 | DEBUG    | src.pipeline:fit_first_level_model:40 - Finish fit ease model
2022-07-16 12:42:24.011 | DEBUG    | src.pipeline:fit_first_level_model:41 - Start fit tensor model


Step 1 of 5
growth of the core: 1.0
Step 2 of 5
growth of the core: 0.2169176699187484
Step 3 of 5
growth of the core: 0.026073396970188303
Step 4 of 5
growth of the core: 0.004295969002684056
Step 5 of 5


2022-07-16 12:45:09.751 | DEBUG    | src.pipeline:fit_first_level_model:43 - Finish fit tensor model


growth of the core: 0.001563181235967085
Done


100%|██████████████████████████████████| 2235/2235 [00:08<00:00, 248.50it/s]
2022-07-16 12:45:26.962 | DEBUG    | src.pipeline:fit:27 - Start fit catboost
2022-07-16 12:46:15.484 | DEBUG    | src.pipeline:fit:33 - Finish fit catboost


In [338]:
data_for_predict = pd.concat([stage2_predict, stage2_train], axis=0, ignore_index=True, copy=False)

In [339]:
data_for_predict[['rating']].value_counts()

rating
4.0       277903
3.0       200747
3.5       138554
5.0       106096
4.5        86041
2.0        75856
2.5        58426
1.0        31027
1.5        18334
0.5        14169
dtype: int64

In [340]:
recommend = model.predict(data_for_predict)

100%|██████████████████████████████████| 2235/2235 [00:02<00:00, 981.67it/s]


In [341]:
model.get_metrics(recommend, stage2_holdout)

(0.040268456375838924, 0.18970917225950784)

In [10]:
n_items

9857

In [11]:
max(stage2_predict.movieid)


8900

In [12]:

max(stage2_train.movieid)

9854

In [16]:
from src import tune_params_and_fit
model, ease_param_best, tensor_param_best, catboost_param_best = tune_params_and_fit(
        stage1_train, stage2_predict, stage2_train, stage2_holdout, n_items, num_params=2)

  0%|                                     | 0/2 [00:00<?, ?trial/s, best loss=?]

2022-07-18 23:52:36.156 | DEBUG    | src.pipeline:fit_first_level_model:36 - Start fit ease model
2022-07-18 23:52:36.688 | INFO     | src.EASEModel:fit:18 - start fitting
2022-07-18 23:52:36.689 | INFO     | src.EASEModel:fit:20 - normalize
2022-07-18 23:52:36.953 | INFO     | src.EASEModel:fit:25 - gram matrix
2022-07-18 23:52:43.563 | INFO     | src.EASEModel:fit:32 - invert
2022-07-18 23:53:00.925 | DEBUG    | src.pipeline:fit_first_level_model:40 - Finish fit ease model
2022-07-18 23:53:00.926 | DEBUG    | src.pipeline:fit_first_level_model:41 - Start fit tensor model


Step 1 of 1                                                                     
growth of the core: 1.0                                                         
Done                                                                            
  0%|                                     | 0/2 [01:10<?, ?trial/s, best loss=?]

2022-07-18 23:53:46.441 | DEBUG    | src.pipeline:fit_first_level_model:43 - Finish fit tensor model
  0%|          | 0/2235 [00:00<?, ?it/s]

  1%|1         | 28/2235 [00:00<00:07, 275.93it/s]

  3%|2         | 57/2235 [00:00<00:07, 280.52it/s]

  4%|3         | 86/2235 [00:00<00:07, 281.76it/s]

  5%|5         | 115/2235 [00:00<00:07, 275.33it/s]

  6%|6         | 143/2235 [00:00<00:07, 264.28it/s]

  8%|7         | 170/2235 [00:00<00:08, 257.92it/s]

  9%|8         | 196/2235 [00:00<00:07, 256.93it/s]

 10%|#         | 224/2235 [00:00<00:07, 261.72it/s]

 11%|#1        | 252/2235 [00:00<00:07, 264.98it/s]

 13%|#2        | 281/2235 [00:01<00:07, 271.46it/s]

 14%|#3        | 309/2235 [00:01<00:07, 272.16it/s]

 15%|#5        | 337/2235 [00:01<00:07, 267.10it/s]

 16%|#6        | 365/2235 [00:01<00:06, 270.44it/s]

 18%|#7        | 393/2235 [00:01<00:06, 271.37it/s]

 19%|#8        | 421/2235 [00:01<00:06, 270.44it/s]

 20%|##        | 449/2235 [00:01<00:06, 264.31it/s]

 21%|##1    


 65%|######4   | 1450/2235 [00:06<00:03, 236.82it/s]

 66%|######5   | 1475/2235 [00:06<00:03, 239.98it/s]

 67%|######7   | 1500/2235 [00:06<00:03, 242.72it/s]

 68%|######8   | 1526/2235 [00:06<00:02, 246.16it/s]

 69%|######9   | 1552/2235 [00:06<00:02, 249.49it/s]

 71%|#######   | 1577/2235 [00:06<00:02, 245.77it/s]

 72%|#######1  | 1602/2235 [00:06<00:02, 241.99it/s]

 73%|#######2  | 1627/2235 [00:06<00:02, 240.00it/s]

 74%|#######3  | 1652/2235 [00:06<00:02, 237.83it/s]

 75%|#######4  | 1676/2235 [00:07<00:02, 236.28it/s]

 76%|#######6  | 1700/2235 [00:07<00:02, 232.82it/s]

 77%|#######7  | 1724/2235 [00:07<00:02, 232.04it/s]

 78%|#######8  | 1748/2235 [00:07<00:02, 233.66it/s]

 79%|#######9  | 1773/2235 [00:07<00:01, 236.79it/s]

 80%|########  | 1799/2235 [00:07<00:01, 241.20it/s]

 82%|########1 | 1824/2235 [00:07<00:01, 242.66it/s]

 83%|########2 | 1849/2235 [00:07<00:01, 241.35it/s]

 84%|########3 | 1874/2235 [00:07<00:01, 239.22it/s]

 85%|########4 | 1898/2235 

...

2022-07-18 23:57:38.173 | INFO     | src.EASEModel:fit:18 - start fitting
2022-07-18 23:57:38.174 | INFO     | src.EASEModel:fit:20 - normalize
2022-07-18 23:57:38.458 | INFO     | src.EASEModel:fit:25 - gram matrix
2022-07-18 23:57:45.105 | INFO     | src.EASEModel:fit:32 - invert
2022-07-18 23:58:02.812 | DEBUG    | src.pipeline:fit_first_level_model:40 - Finish fit ease model
2022-07-18 23:58:02.813 | DEBUG    | src.pipeline:fit_first_level_model:41 - Start fit tensor model


Step 1 of 1


2022-07-18 23:58:51.878 | DEBUG    | src.pipeline:fit_first_level_model:43 - Finish fit tensor model


growth of the core: 1.0
Done


100%|██████████████████████████████████████| 2235/2235 [00:07<00:00, 280.89it/s]
2022-07-18 23:59:09.133 | DEBUG    | src.pipeline:fit:27 - Start fit catboost
2022-07-18 23:59:12.088 | DEBUG    | src.pipeline:fit:33 - Finish fit catboost


In [18]:
ease_param_best, tensor_param_best, catboost_param_best

({'reg_weight': 425},
 {'core_shape': 83},
 {'colsample_bylevel': 0.6,
  'depth': 13,
  'min_child_samples': 287,
  'n_estimators': 62,
  'reg_lambda': 20,
  'subsample': 0.1})

In [19]:
model.fit_first_level_model(final_training)

2022-07-18 23:59:50.200 | DEBUG    | src.pipeline:fit_first_level_model:36 - Start fit ease model
2022-07-18 23:59:50.936 | INFO     | src.EASEModel:fit:18 - start fitting
2022-07-18 23:59:50.936 | INFO     | src.EASEModel:fit:20 - normalize
2022-07-18 23:59:51.347 | INFO     | src.EASEModel:fit:25 - gram matrix
2022-07-19 00:00:02.100 | INFO     | src.EASEModel:fit:32 - invert
2022-07-19 00:00:19.897 | DEBUG    | src.pipeline:fit_first_level_model:40 - Finish fit ease model
2022-07-19 00:00:19.898 | DEBUG    | src.pipeline:fit_first_level_model:41 - Start fit tensor model


Step 1 of 1


2022-07-19 00:01:22.347 | DEBUG    | src.pipeline:fit_first_level_model:43 - Finish fit tensor model


growth of the core: 1.0
Done


In [20]:
recommend = model.predict(final_testset)

100%|██████████████████████████████████████| 5198/5198 [00:10<00:00, 488.72it/s]


In [21]:
model.get_metrics(recommend, final_holdout)

(0.06271642939592151, 0.3053097345132743)